In [1]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/blend2/blend-2/1866.csv
/kaggle/input/blend2/blend-2/1856.csv
/kaggle/input/blend2/blend-2/1872.csv
/kaggle/input/blend2/blend-2/1855.csv
/kaggle/input/blend1dir/blend/1856.csv
/kaggle/input/blend1dir/blend/1872.csv
/kaggle/input/blend1dir/blend/1855.csv
/kaggle/input/lish-moa/train_targets_scored.csv
/kaggle/input/lish-moa/train_targets_nonscored.csv
/kaggle/input/lish-moa/train_features.csv
/kaggle/input/lish-moa/sample_submission.csv
/kaggle/input/lish-moa/test_features.csv


In [2]:
submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

public_ids = list(submission['sig_id'].values)

submission.shape


(3982, 207)

In [3]:
test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')

test_ids = list(test['sig_id'].values)

private_ids = list(set(test_ids)-set(public_ids))

len(private_ids)

0

In [4]:
kernels = {
    'kernel':['../input/blend2/blend-2/1855.csv',
             '../input/blend2/blend-2/1856.csv',
             '../input/blend2/blend-2/1866.csv',
             '../input/blend2/blend-2/1872.csv'],
    'score':[0.01855,
            0.01856,
            0.01866,
            0.01872]
}

In [5]:
kernels=pd.DataFrame(kernels)
kernels

,kernel,score
0,../input/blend2/blend-2/1855.csv,0.01855
1,../input/blend2/blend-2/1856.csv,0.01856
2,../input/blend2/blend-2/1866.csv,0.01866
3,../input/blend2/blend-2/1872.csv,0.01872


In [12]:
## sonu 

filenames= kernels.kernel.values

predictions = [pd.read_csv(f) for f in filenames]

In [13]:
columns = list(submission.columns)
columns.remove('sig_id')

In [14]:
y_pred = pd.DataFrame()
y_pred['sig_id'] = predictions[0]['sig_id']

for column in columns:
    column_data = []
    for i in range(len(predictions)):
        column_data.append(predictions[i][column])
    y_pred[column] = np.mean(column_data, axis=0)

y_pred.shape

(3982, 207)

In [15]:
scores = kernels['score']

sum_scores = sum(scores)

weights = [x / sum_scores for x in scores]
weights

[0.24902671499530138,
 0.24916096120284603,
 0.2505034232782924,
 0.2513089005235602]

In [16]:
weighted_y_pred = pd.DataFrame()
weighted_y_pred['sig_id'] = predictions[0]['sig_id']

for column in columns:
    column_data = []
    for i in range(len(predictions)):
        column_data.append(predictions[i][column] * weights[i])
    weighted_y_pred[column] = np.sum(column_data, axis=0)

weighted_y_pred.shape

(3982, 207)

In [17]:
y_pred = weighted_y_pred

In [20]:
submission = pd.DataFrame(index = public_ids + private_ids, columns=columns)
submission.index.name = 'sig_id'

submission[:] = 0

submission.loc[y_pred.sig_id,:] = y_pred[columns].values

submission.loc[test[test.cp_type == 'ctl_vehicle'].sig_id] = 0

submission.to_csv('submission.csv', index=True)

In [19]:
submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
sig_id,,,,,,,,,,,,,,,,,,,,,
id_0004d9e33,0.000843,0.001034,0.001913,0.016337,0.019536,0.004414,0.001755,0.005807,0.000246,0.012314,...,0.000756,0.001588,0.002960,0.001332,0.000607,0.000564,0.000790,0.001818,0.005329,0.001444
id_001897cda,0.000372,0.001003,0.001661,0.002641,0.002280,0.002385,0.003687,0.010097,0.006820,0.013487,...,0.000750,0.001041,0.003827,0.000410,0.010796,0.000551,0.009157,0.000865,0.002212,0.003081
id_002429b5b,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
id_00276f245,0.000649,0.000765,0.001845,0.010970,0.012014,0.004361,0.002755,0.004662,0.000412,0.016352,...,0.000538,0.001072,0.003104,0.017958,0.006723,0.000516,0.001616,0.001771,0.001295,0.002734
id_0027f1083,0.001526,0.001382,0.001826,0.015079,0.021282,0.003556,0.005132,0.002165,0.000428,0.011163,...,0.000727,0.000547,0.004434,0.002377,0.000950,0.000608,0.001643,0.001849,0.000337,0.001446
